**goal is to trim 15 510 geographical articles, to charaterize each headphrase by the Geographical Entities (GE) mentionned.**

In [1]:
import sys
sys.path.append('/home/antoine/Documents/GitHub/linkencyclo')
datapath = '/home/antoine/Documents/GitHub/datas/'

from EncycloObject import Article, Book
import pickle
import stanza

We load a `Book` instance :

In [2]:
with open(datapath+'mybook_19062024.pkl', 'rb') as f:
    mybook = pickle.load(f)
print(mybook)

Book with 15510


Example of the Article instance with hash `7/2925/GRENOBLE`

In [3]:
grenoble = mybook._reach_article(headword='grenoble')
grenoble.artfl, grenoble.hash, grenoble.text

('https://artflsrv04.uchicago.edu/philologic4.7/encyclopedie0922/navigate/7/2925',
 '7/2925/GRENOBLE',
 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, & un parlement érigé en 1493 par Louis XI. qui n\'étoit encore que dauphin ;  mais son pere ratifia cette érection deux ans après.\nGrenoble est sur l\'Isere, à onze lieues S O. de Chambéri, quarante-deux N. O. de Turin, seize S. E. de Vienne, cent vingt-quatre S. O. de Paris. Long. suivant Harris, 23d. 31\'. 15". suivant Cassini, 23d. 14\'. 15". latit 45d. 11\'.\nCette ville reçut le nom de Gratianopolis de l\'empereur  Gratien fils de Valentinien I. car elle s\'appelloit  auparavant Cularo ; & c\'est sous ce nom qu\'il en est parlé dans une lettre de Plancus à Cicéron, epist. xxiij. Long-tems après, les Romains l\'érigerent en cité: dans le cinquieme siecle, elle fut assujettie par les Bourguignons, & dans le sixieme par les François Mérovingiens ; ensuite elle a 

# Part-of-Speech parsing
We rely on stanza parser to tag each token with a POS. This will help us to normalize the GE phrases : going from
- `ancienne ville` to `ville`
- `royaume de France` to `france`
- `St. Etienne` to `saint etienne`

Stanza doc are stored in the `.parsed` attribute

In [4]:
nlpstanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt, pos')

2024-08-21 18:32:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-08-21 18:32:22 INFO: Downloaded file to /home/antoine/stanza_resources/resources.json
2024-08-21 18:32:22 INFO: Loading these models for language: fr (French):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2024-08-21 18:32:22 INFO: Using device: cpu
2024-08-21 18:32:22 INFO: Loading: tokenize
2024-08-21 18:32:23 INFO: Loading: mwt
2024-08-21 18:32:23 INFO: Loading: pos
2024-08-21 18:32:23 INFO: Done loading processors!


In [5]:
for article in [grenoble]:
    article.parsed = nlpstanza(article.text)
    #break

# Named Entity Recognition
We apply the BERT model to identify GE in each article.

From
> 7/2925/GRENOBLE 
> 
> 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, ...'

We want to store in the `article.ner` attribute the following list :
```python
[
    {'entity_group': 'Head', 'score': 0.96214443, 'word': 'GRENOBLE, Gratianopolis', 'start': 0, 'end': 23},
    {'entity_group': 'Domain_mark', 'score': 0.9491153, 'word': 'Géogr.', 'start': 18, 'end': 24},
    {'entity_group': 'NC_Spatial', 'score': 0.9601506, 'word': 'ancienne ville', 'start': 26, 'end': 40},
    {'entity_group': 'NP_Spatial', 'score': 0.96639144, 'word': 'France', 'start': 44, 'end': 50},
    {'entity_group': 'NC_Spatial', 'score': 0.96316826, 'word': 'capitale', 'start': 52, 'end': 60},
    {'entity_group': 'NP_Spatial', 'score': 0.9662918, 'word': 'Dauphiné', 'start': 64, 'end': 72},
    {'entity_group': 'NC_Spatial', 'score': 0.96156704, 'word': 'un évêché suffragant', 'start': 79, 'end': 99},
    {'entity_group': 'NP_Spatial', 'score': 0.9662691, 'word': 'Vienne', 'start': 103, 'end': 109},
    ...
]
```


In [6]:
# loading stopwords from Solr
with open('solr_stopwords.txt', 'r') as f:
    stopwords = f.read().split('\n') 

In [7]:
from transformers import pipeline
bert = pipeline("token-classification", model="GEODE/bert-base-french-cased-edda-ner",
                aggregation_strategy="simple")

/home/antoine/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/antoine/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
for article in [grenoble]:
    article.ner = article._apply_pipeline(bert)
    #break

We then add NER tags to the Stanza doc thanks to the method `Article._enrich_stanzadoc`
- native `Token` instances of the Stanza doc receive the NER tags
- native `Span` instances of the Stanza doc receive the contiguous merged NER tags

In [18]:
for article in [grenoble]:
    article.ner = article._apply_pipeline(bert)
    ncs, nps = article._enrich_stanzadoc(stopwords=stopwords)

In [20]:
grenoble.ncs == ncs, grenoble.nps == nps

(True, True)